In [ ]:
!pip install transformers==4.36.2

In [ ]:
import multiprocessing
multiprocessing.cpu_count()

2

In [ ]:
import numpy as np
import pandas as pd
import time

In [ ]:
# Fetch Clean data
## If you are using the data by mounting the google drive, use the following :
from google.colab import drive
drive.mount('/content/gdrive',  force_remount=False)

%cd /content/gdrive/MyDrive/FinalThesis/Final_Data/

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive/FinalThesis/Final_Data


In [ ]:
#import torch
#from torch.utils.data.dataset import Dataset
import scipy
#from transformers import pipeline
#from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, BertConfig

In [ ]:
#Load Twitter Dataset
start = time.time()
tweet_clean = pd.read_csv("clean_data_20-01-2024.csv")
t = time.time() - start
print('Time taken to load Twitter Clean Data:',t)

Time taken to load Twitter Clean Data: 2.6272122859954834


In [ ]:
tweet_clean.dtypes

date_time     object
clean_text    object
dtype: object

In [ ]:
tweet_clean['date_time'] = pd.to_datetime(tweet_clean['date_time'])
tweet_clean['clean_text'] = tweet_clean['clean_text'].astype(str)

In [ ]:
tweet_clean.head()

,date_time,clean_text
0,2021-02-10 23:59:04,blue ridge bank shares halted nyse bitcoin atm...
1,2021-02-10 23:58:48,today thats thursday take friend btc wallet se...
2,2021-02-10 23:54:48,guys evening read article btc would like share
3,2021-02-10 23:54:33,btc big chance billion price bitcoin fx btc cr...
4,2021-02-10 23:54:06,network secured nodes today soon biggest bears...


In [ ]:
tweet_clean.shape

(1048570, 2)

In [ ]:
dataByMonth= tweet_clean[(tweet_clean['date_time'].dt.month == 8)& ((tweet_clean['date_time'].dt.day >= 26) & (tweet_clean['date_time'].dt.day <= 31))]
dataByMonth = dataByMonth.dropna()


In [ ]:
#ddf.npartitions
#ddf=tweet_clean.head(100)
ddf=dataByMonth

In [ ]:
finbert_tokenizer = BertTokenizer.from_pretrained('ProsusAI/finbert')
finbert_model = BertForSequenceClassification.from_pretrained('ProsusAI/finbert')

In [ ]:
labels = finbert_model.config.id2label.values()
from scipy import special
def getSenimentScore(clean_text):
  clean_data = finbert_tokenizer(clean_text, padding=True, truncation=True, return_tensors='pt')
  logits = finbert_model(**clean_data).logits
  scores = {k: v for k, v in zip(labels, scipy.special.softmax(logits.detach().numpy().squeeze()))}
  return scores

In [ ]:
#compound_score = (-scores['negative'] + scores['positive'])

In [ ]:
from tqdm import tqdm

In [ ]:
start = time.time()
print(start)
tqdm.pandas(desc='Calculating Sentiment Score ---')
ddf['compound_score'] = ddf['clean_text'].progress_apply(lambda x: getSenimentScore(x))
t = time.time() - start
print("Time taken to calculate FinBert sentiment score :",t)

1714314658.447249


Calculating Sentiment Score ---: 100%|██████████| 100/100 [00:20<00:00,  4.79it/s]

Time taken to calculate FinBert sentiment score : 20.916815996170044



<ipython-input-41-902a24bb1196>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ddf['compound_score'] = ddf['clean_text'].progress_apply(lambda x: getSenimentScore(x))


In [ ]:
ddf.head()

,date_time,clean_text,compound_score
0,2021-02-10 23:59:04,blue ridge bank shares halted nyse bitcoin atm...,"{'positive': 0.025041353, 'negative': 0.304201..."
1,2021-02-10 23:58:48,today thats thursday take friend btc wallet se...,"{'positive': 0.034128867, 'negative': 0.026364..."
2,2021-02-10 23:54:48,guys evening read article btc would like share,"{'positive': 0.060913764, 'negative': 0.012854..."
3,2021-02-10 23:54:33,btc big chance billion price bitcoin fx btc cr...,"{'positive': 0.040152393, 'negative': 0.020609..."
4,2021-02-10 23:54:06,network secured nodes today soon biggest bears...,"{'positive': 0.5442933, 'negative': 0.01050748..."


In [ ]:
today_date = time.strftime("%d-%m-%Y")
excel_filename = "finbert_compound_score_"+today_date+".csv"
print(excel_filename)
ddf.to_csv(excel_filename, index=False, header=True)

finbert_compound_score_28-04-2024.csv
